In [1]:
import trimesh

In [2]:
scene = trimesh.load("/project/uva_cv_lab/xuweic/Diffusion4D/rendering/obj_v1/glbs/000-003/ae3ce147938d47788cf9d5f29d32fa66.glb")

In [3]:
for geometry_name, geometry in scene.geometry.items():
    print(f"Geometry: ", {geometry_name})
    vertices = geometry.vertices
    print("vertices:", vertices)

Geometry:  {'Object_0'}
vertices: [[0.60485202 4.7158699  3.61233997]
 [0.64307702 4.71852016 3.61870003]
 [0.62980598 4.66430998 3.59308004]
 ...
 [0.62055999 3.98623991 3.76114035]
 [0.60938501 4.00989008 3.71111965]
 [0.565925   3.96371007 3.75281   ]]
Geometry:  {'Object_1'}
vertices: [[-0.52279991  3.95558405  3.02643228]
 [-0.54373085  3.91233635  3.01267648]
 [-0.55486232  3.95780706  3.03304195]
 ...
 [-0.52660209  3.3634212   3.11281824]
 [-0.53597546  3.34358382  3.15477443]
 [-0.49014866  3.32468629  3.14778709]]
Geometry:  {'Object_2'}
vertices: [[-0.95352674  4.21678877  1.94444633]
 [-1.09731901  4.82896519  2.03127694]
 [-1.03397417  4.48857069  2.0807147 ]
 ...
 [-7.24848032 15.63807297 -3.29787874]
 [-7.15951109 15.47039986 -3.70452023]
 [-7.08069944 15.38291454 -3.66143179]]
Geometry:  {'Object_3'}
vertices: [[-1.53574157  5.42876101  1.50156128]
 [-1.46158493  5.21224594  1.49345028]
 [-1.37596214  5.07326841  1.82479405]
 ...
 [-4.7860446   9.5331707  -9.94199181]
 

In [11]:
import numpy as np
import json
import base64
from pygltflib import GLTF2


def extract_accessor_data(gltf, accessor):
    """
    Extract data from an accessor in a GLTF file.

    Args:
        gltf (GLTF2): The GLTF file object.
        accessor (Accessor): The accessor to extract data from.

    Returns:
        np.ndarray: Extracted data as a NumPy array.
    """
    buffer_view = gltf.bufferViews[accessor.bufferView]
    buffer = gltf.buffers[buffer_view.buffer]

    # Map GLTF accessor types to their dimensionality
    type_mapping = {
        "SCALAR": 1,
        "VEC2": 2,
        "VEC3": 3,
        "VEC4": 4,
        "MAT2": 4,
        "MAT3": 9,
        "MAT4": 16,
    }
    num_components = type_mapping.get(accessor.type, 1)

    # Calculate offsets and sizes
    byte_offset = (accessor.byteOffset or 0) + (buffer_view.byteOffset or 0)
    byte_stride = buffer_view.byteStride or num_components * np.dtype(np.float32).itemsize
    byte_length = accessor.count * num_components * np.dtype(np.float32).itemsize

    # Handle buffer data
    if buffer.uri is None:
        # Buffer data is embedded in the GLB binary chunk
        binary_blob = gltf.binary_blob()
        if binary_blob is None:
            raise ValueError("Buffer data is missing or invalid.")
        raw_data = binary_blob[byte_offset:byte_offset + byte_length]
    elif buffer.uri.startswith("data:"):
        # Decode embedded base64 data
        _, base64_data = buffer.uri.split(",", 1)
        raw_data = base64.b64decode(base64_data)
    else:
        # Load data from an external file
        with open(buffer.uri, "rb") as f:
            f.seek(byte_offset)
            raw_data = f.read(byte_length)

    # Convert raw binary data to NumPy array
    data = np.frombuffer(raw_data, dtype=np.float32)
    return data.reshape(accessor.count, num_components)


def inspect_glb(filepath):
    """
    Inspect a GLB file for nodes and animations.

    Args:
        filepath (str): Path to the GLB file.

    Returns:
        dict: Information about the GLB file, including nodes and animations.
    """
    gltf = GLTF2().load(filepath)

    # Extract node information
    node_info = []
    for idx, node in enumerate(gltf.nodes):
        node_info.append({
            "index": idx,
            "name": node.name,
            "mesh": node.mesh,
            "translation": node.translation,
            "rotation": node.rotation,
            "scale": node.scale,
        })

    # Extract animation information
    anim_info = []
    for animation in gltf.animations or []:
        anim = {
            "name": animation.name,
            "channels": [],
        }
        for channel in animation.channels:
            sampler = animation.samplers[channel.sampler]
            input_accessor = gltf.accessors[sampler.input]
            output_accessor = gltf.accessors[sampler.output]

            try:
                input_data = extract_accessor_data(gltf, input_accessor)
                output_data = extract_accessor_data(gltf, output_accessor)
            except ValueError as e:
                print(f"Error extracting data for channel {channel.target.path}: {e}")
                continue

            anim["channels"].append({
                "node": channel.target.node,
                "path": channel.target.path,
                "keyframes": input_data.tolist(),
                "values": output_data.tolist(),
            })
        anim_info.append(anim)

    return {"nodes": node_info, "animations": anim_info}

In [13]:
filepath = "/project/uva_cv_lab/xuweic/Diffusion4D/rendering/obj_v1/glbs/000-003/ae3ce147938d47788cf9d5f29d32fa66.glb"
info = inspect_glb(filepath)

# Print node and animation information
print("Nodes:")
for node in info["nodes"]:
    print(json.dumps(node, indent=2))

print("\nAnimations:")
for animation in info["animations"]:
    print(json.dumps(animation, indent=2))

Nodes:
{
  "index": 0,
  "name": "Sketchfab_model",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 1,
  "name": "root",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 2,
  "name": "GLTF_SceneRootNode",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 3,
  "name": "_0",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 4,
  "name": "group_MariposaNoTransformar_1",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 5,
  "name": "Butterfly_Mesh_199",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 6,
  "name": "eye_L_216",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 7,
  "name": "eye_L:PIV_217",
  "mesh": null,
  "translation": null,
  "rotation": null,
  "scale": null
}
{
  "index": 8,
  "name": "Ob

In [46]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from pygltflib import GLTF2


def extract_accessor_data(gltf, accessor):
    """
    Extract data from a GLTF accessor.
    """
    buffer_view = gltf.bufferViews[accessor.bufferView]
    buffer = gltf.buffers[buffer_view.buffer]

    # Map GLTF types to dimensions
    type_mapping = {
        "SCALAR": 1,
        "VEC2": 2,
        "VEC3": 3,
        "VEC4": 4,
        "MAT2": 4,
        "MAT3": 9,
        "MAT4": 16,
    }
    num_components = type_mapping[accessor.type]

    # Compute offsets and sizes
    byte_offset = (accessor.byteOffset or 0) + (buffer_view.byteOffset or 0)
    byte_length = accessor.count * num_components * np.dtype(np.float32).itemsize

    if buffer.uri is None:
        # Binary blob for embedded buffers
        binary_blob = gltf.binary_blob()
        if binary_blob is None:
            raise ValueError("Buffer data is missing.")
        raw_data = binary_blob[byte_offset:byte_offset + byte_length]
    elif buffer.uri.startswith("data:"):
        # Decode embedded base64 data
        _, base64_data = buffer.uri.split(",", 1)
        raw_data = base64.b64decode(base64_data)
    else:
        # External buffer
        with open(buffer.uri, "rb") as f:
            f.seek(byte_offset)
            raw_data = f.read(byte_length)

    # Convert to NumPy array
    data = np.frombuffer(raw_data, dtype=np.float32)
    return data.reshape(accessor.count, num_components)


def get_static_points(gltf, mesh_index):
    """
    Extract static point cloud from a mesh.
    """
    mesh = gltf.meshes[mesh_index]
    positions_accessor = gltf.accessors[mesh.primitives[0].attributes.POSITION]
    return extract_accessor_data(gltf, positions_accessor)


def initialize_points_from_nodes(gltf):
    """
    Initialize points from gltf.nodes, applying transformations at t=0.
    """
    static_points_by_node = {}
    for node_index, node in enumerate(gltf.nodes):
        if node.mesh is not None:
            # Get static points from the mesh
            static_points = get_static_points(gltf, node.mesh)
            
            # Apply initial transformations
            translation = np.array(node.translation or [0, 0, 0])
            rotation = np.array(node.rotation or [0, 0, 0, 1])
            scale = np.array(node.scale or [1, 1, 1])

            # Apply rotation and scale
            rot_matrix = R.from_quat(rotation).as_matrix()
            transformed_points = (static_points @ rot_matrix.T) * scale + translation

            static_points_by_node[node_index] = transformed_points
    return static_points_by_node


def interpolate_keyframes(keyframes, values, t):
    """
    Interpolate keyframe values for a given time t.
    """
    if t <= keyframes[0]:
        return np.array(values[0])
    if t >= keyframes[-1]:
        return np.array(values[-1])

    idx = np.searchsorted(keyframes, t) - 1
    t0, t1 = keyframes[idx], keyframes[idx + 1]
    v0, v1 = np.array(values[idx]), np.array(values[idx + 1])

    factor = (t - t0) / (t1 - t0)
    return v0 + factor * (v1 - v0)


def apply_animation(node, t, gltf, animations):
    """
    Apply animation transformations for a specific node at time t.
    """
    translation = np.array(node.translation or [0, 0, 0])
    rotation = np.array(node.rotation or [0, 0, 0, 1])
    scale = np.array(node.scale or [1, 1, 1])

    for animation in animations or []:
        for channel in animation.channels:
            if channel.target.node == node:
                keyframes = np.array(channel.keyframes).flatten()
                values = np.array(channel.values)

                if channel.path == "translation":
                    translation = interpolate_keyframes(keyframes, values, t)
                elif channel.path == "rotation":
                    rotation = interpolate_keyframes(keyframes, values, t)
                elif channel.path == "scale":
                    scale = interpolate_keyframes(keyframes, values, t)

    return translation, rotation, scale


def extract_trajectory_points(filepath, output_path, num_timesteps=100):
    """
    Extract trajectory points for all nodes over time.
    """
    gltf = GLTF2().load(filepath)
    static_points_by_node = initialize_points_from_nodes(gltf)
    trajectories = {}

    # Prepare timesteps
    max_time = num_timesteps
    timesteps = np.linspace(0, max_time, num=num_timesteps)

    # Process trajectories
    for t in timesteps:
        for node_index, node in enumerate(gltf.nodes):
            if node_index not in static_points_by_node:
                continue

            translation, rotation, scale = apply_animation(node, t, gltf, gltf.animations)
            rot_matrix = R.from_quat(rotation).as_matrix()

            # Transform static points
            transformed_points = (static_points_by_node[node_index] @ rot_matrix.T) * scale + translation
            if t == 0:
                trajectories[node_index] = [transformed_points]
            else:
                trajectories[node_index].append(transformed_points)

    # Combine trajectories
    all_trajectories = []
    for node_index, trajectory in trajectories.items():
        all_trajectories.append(np.stack(trajectory, axis=0))

    final_trajectory = np.concatenate(all_trajectories, axis=1)

    # Save trajectory
    np.save(output_path, final_trajectory)
    print(f"Saved trajectory to {output_path}")

In [47]:
filepath = "/project/uva_cv_lab/xuweic/Diffusion4D/rendering/obj_v1/glbs/000-003/ae3ce147938d47788cf9d5f29d32fa66.glb"
output_path = "/project/uva_cv_lab/xuweic/Diffusion4D/rendering/4d_tracking_output/ae3ce147938d47788cf9d5f29d32fa66/trajectory.npy"

extract_vertex_trajectories(filepath, output_path)

In [48]:
import numpy as np

In [49]:
a = np.load("/project/uva_cv_lab/xuweic/Diffusion4D/rendering/4d_tracking_output/ae3ce147938d47788cf9d5f29d32fa66/trajectory.npy")

In [50]:
a.shape

(100, 152928, 3)

In [51]:
a[0, :, :]

array([[0.60485202, 4.7158699 , 3.61233997],
       [0.64307702, 4.71852016, 3.61870003],
       [0.62980598, 4.66430998, 3.59308004],
       ...,
       [0.44816926, 4.13964605, 3.07462025],
       [0.48652929, 4.06316614, 3.05687165],
       [0.43771136, 4.13739014, 3.07768178]])

In [52]:
a[3, :, :]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [34]:
def save_frame_as_ply(trajectory, filepath):
    """
    Saves the first frame of the trajectory as a PLY file.

    Args:
        trajectory (np.ndarray): The trajectory array of shape (t, num_points, 3).
        filepath (str): The file path to save the PLY file.
    """
    first_frame = trajectory[0, :, :]  # Extract the first frame (t=0)
    
    with open(filepath, "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {first_frame.shape[0]}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("end_header\n")
        for point in first_frame:
            f.write(f"{point[0]} {point[1]} {point[2]}\n")

In [35]:
trajectory = np.load("/project/uva_cv_lab/xuweic/Diffusion4D/rendering/4d_tracking_output/ae3ce147938d47788cf9d5f29d32fa66/trajectory.npy")  # Load trajectory array
output_ply_path = "/project/uva_cv_lab/xuweic/Diffusion4D/rendering/4d_tracking_output/ae3ce147938d47788cf9d5f29d32fa66/frame0.ply"  # Define the PLY file path
save_frame_as_ply(trajectory, output_ply_path)  # Save the first frame